In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 定義圖像文件夾路徑
image_folder = r'D:\falldetection\Common use\bodyimage\CNN'

In [2]:
def preprocess_image(image):
    # 調整圖像大小為指定尺寸
    resized_image = cv2.resize(image, (192, 192))
    # 正規化圖像像素值到 [0, 1] 範圍
    normalized_image = resized_image / 255.0
    return normalized_image

In [4]:
image_height = 192
image_width = 192
# 初始化列表來存儲圖像數據和標籤
images = []
labels = []


# 遍歷文件夾中的每張照片
for label, class_name in enumerate(['stand', 'sit', 'lie']):
    class_folder = os.path.join(image_folder, class_name)
    for filename in os.listdir(class_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # 讀取圖像
            image_path = os.path.join(class_folder, filename)
            image = cv2.imread(image_path)
            
            # 對圖像進行預處理，例如調整大小、歸一化等
            processed_image = preprocess_image(image)
            
            # 將預處理後的圖像添加到列表中
            images.append(processed_image)
            
            # 添加對應的標籤
            labels.append(label)
            
# 將圖像列表轉換為numpy數組
images = np.array(images)

# 將標籤列表轉換為numpy數組
labels = np.array(labels)

# 將圖像數據進行訓練/測試集的劃分
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

# 正規化圖像數據
X_train = X_train / 255.0
X_test = X_test / 255.0


In [7]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 建立CNN模型
model = Sequential()

# Add the convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D((2, 2)))

# Drop掉部分神經元避免overfitting
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

# Flatten the feature maps
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [8]:
# 編譯模型 optimizer='adam'會自動調整 learning rate
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# 訓練模型
train_history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
27/27 [==============================] - 18s 625ms/step - loss: 0.9800 - accuracy: 0.5313 - val_loss: 0.8964 - val_accuracy: 0.6056
Epoch 2/20
27/27 [==============================] - 20s 747ms/step - loss: 0.9164 - accuracy: 0.5436 - val_loss: 0.8912 - val_accuracy: 0.6056
Epoch 3/20
27/27 [==============================] - 21s 775ms/step - loss: 0.9219 - accuracy: 0.5454 - val_loss: 0.8831 - val_accuracy: 0.6056
Epoch 4/20
27/27 [==============================] - 21s 768ms/step - loss: 0.9402 - accuracy: 0.5417 - val_loss: 0.8832 - val_accuracy: 0.6056
Epoch 5/20
27/27 [==============================] - 21s 782ms/step - loss: 0.8554 - accuracy: 0.5936 - val_loss: 0.8787 - val_accuracy: 0.6056
Epoch 6/20
27/27 [==============================] - 21s 790ms/step - loss: 0.9439 - accuracy: 0.5429 - val_loss: 0.8437 - val_accuracy: 0.6056
Epoch 7/20
27/27 [==============================] - 22s 812ms/step - loss: 0.7954 - accuracy: 0.5713 - val_loss: 0.6249 - val_accuracy: 0.6778

In [10]:
# 儲存模型
model.save(r'D:\falldetection\trainedModelforCNN\theFirst')

INFO:tensorflow:Assets written to: D:\falldetection\trainedModelforCNN\theFirst\assets
